In [3]:
import numpy as np
import pandas as pd
import string
#import xgboost as xgb
import io
import nltk
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('stopwords')
stopwords = stopwords.words('english')
stemmer = SnowballStemmer('english')

from textblob import TextBlob
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/matiascano/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matiascano/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/matiascano/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/matiascano/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
sia = SentimentIntensityAnalyzer()
def return_sia_compound_values(text):
    return sia.polarity_scores(text)['compound']

In [5]:
def remove_stopword(text):
    new_text = []
    for e in text:
        if e not in stopwords and e.isalpha():
            new_text.append(e)
    text = new_text
    return " ".join(new_text)

def stemm(text):
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)

def contains_punctuation(text):
    punctuation = set(string.punctuation)
    for character in text:
        if character in punctuation:
            return True
    return False

def amount_of_punctuation(text):
    punctuation = set(string.punctuation)
    amount = 0
    for character in text:
        if character in punctuation: amount += 1
    return amount

def get_adjectives(text):
    blob = TextBlob(text)
    return len([word for (word,tag) in blob.tags if tag.startswith("JJ")])

def get_nouns(text):
    blob = TextBlob(text)
    return len([word for (word,tag) in blob.tags if tag.startswith("NN")])

def get_verbs(text):
    blob = TextBlob(text)
    return len([word for (word,tag) in blob.tags if tag.startswith("VB")])

def get_adverbs(text):
    blob = TextBlob(text)
    return len([word for (word,tag) in blob.tags if tag.startswith("RB")])

In [6]:
tweets = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [7]:
tweets.drop_duplicates(subset = 'text', keep = False, inplace = True)
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7434 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7434 non-null   int64 
 1   keyword   7378 non-null   object
 2   location  4982 non-null   object
 3   text      7434 non-null   object
 4   target    7434 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 348.5+ KB


In [8]:
test['text'].duplicated().value_counts()

False    3243
True       20
Name: text, dtype: int64

### Feature Engineering

In [9]:
tweets_metrics = tweets[['id','text','target']]
tweets_metrics['text_without_stopwords'] = tweets_metrics['text'].str.split()
tweets_metrics['text_without_stopwords'] = tweets_metrics['text_without_stopwords'].apply(remove_stopword)

tweets_metrics['length'] = tweets_metrics['text'].apply(lambda x: len(x))
tweets_metrics['avg_word_length'] = tweets_metrics['text'].str.split().apply(lambda x: [len(y) for y in x]).transform(lambda x: np.mean(x))
tweets_metrics['amount_of_words'] = tweets_metrics['text'].str.split().transform(lambda x: len(x))
unique_words_by_tweet = tweets_metrics['text'].transform(lambda x: x.split()).transform(lambda x: pd.Series(x).unique()).transform(lambda x: len(x))
tweets_metrics['amount_of_unique_words'] = unique_words_by_tweet
tweets_metrics['sentiment'] = tweets_metrics['text'].apply(lambda x: return_sia_compound_values(x))
tweets_metrics['stopwords_count'] = tweets_metrics['text'].apply(lambda x: len([word for word in str(x).lower().split() if word in stopwords]))
tweets_metrics['punctuation_count'] = tweets_metrics['text'].apply(lambda x: amount_of_punctuation(x))
mentions = tweets_metrics['text'].str.findall(r'@.\S*?(?=\s|[:]|$)').to_frame()
tweets_metrics['mentions_count'] = mentions['text'].apply(lambda x: len(x))
hashtags = tweets_metrics['text'].str.findall(r'#[^?\s].*?(?=\s|$)')
tweets_metrics['hashtags_count'] = hashtags.apply(lambda x: len(x))
tweets_metrics['longest_word_length_without_stopwords'] = tweets_metrics['text_without_stopwords'].apply(lambda x: ([len(word) for word in str(x).lower().split() if not word.startswith('http')])).apply(lambda x: max(x) if len(x) > 0 else 0)
tweets_metrics['stopword_word_ratio'] = tweets_metrics['stopwords_count'] / tweets_metrics['amount_of_words']

tweets_metrics['adjectives_count'] = tweets_metrics['text'].apply(get_adjectives)
tweets_metrics['nouns_count'] = tweets_metrics['text'].apply(get_nouns)
tweets_metrics['verbs_count'] = tweets_metrics['text'].apply(get_verbs)
tweets_metrics['adverbs_count'] = tweets_metrics['text'].apply(get_adverbs)

/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
tweets_metrics.to_csv('train_features.csv', index=False)

In [11]:
test_metrics = test[['id','text']]
test_metrics['text_without_stopwords'] = test_metrics['text'].str.split()
test_metrics['text_without_stopwords'] = test_metrics['text_without_stopwords'].apply(remove_stopword)

test_metrics['length'] = test_metrics['text'].apply(lambda x: len(x))
test_metrics['avg_word_length'] = test_metrics['text'].str.split().apply(lambda x: [len(y) for y in x]).transform(lambda x: np.mean(x))
test_metrics['amount_of_words'] = test_metrics['text'].str.split().transform(lambda x: len(x))
unique_words_by_tweet = test_metrics['text'].transform(lambda x: x.split()).transform(lambda x: pd.Series(x).unique()).transform(lambda x: len(x))
test_metrics['amount_of_unique_words'] = unique_words_by_tweet
test_metrics['sentiment'] = test_metrics['text'].apply(lambda x: return_sia_compound_values(x))
test_metrics['stopwords_count'] = test_metrics['text'].apply(lambda x: len([word for word in str(x).lower().split() if word in stopwords]))
test_metrics['punctuation_count'] = test_metrics['text'].apply(lambda x: amount_of_punctuation(x))
mentions = test_metrics['text'].str.findall(r'@.\S*?(?=\s|[:]|$)').to_frame()
test_metrics['mentions_count'] = mentions['text'].apply(lambda x: len(x))
hashtags = test_metrics['text'].str.findall(r'#[^?\s].*?(?=\s|$)')
test_metrics['hashtags_count'] = hashtags.apply(lambda x: len(x))
test_metrics['longest_word_length_without_stopwords'] = test_metrics['text_without_stopwords'].apply(lambda x: ([len(word) for word in str(x).lower().split() if not word.startswith('http')])).apply(lambda x: max(x) if len(x) > 0 else 0)
test_metrics['stopword_word_ratio'] = test_metrics['stopwords_count'] / test_metrics['amount_of_words']

test_metrics['adjectives_count'] = test_metrics['text'].apply(get_adjectives)
test_metrics['nouns_count'] = test_metrics['text'].apply(get_nouns)
test_metrics['verbs_count'] = test_metrics['text'].apply(get_verbs)
test_metrics['adverbs_count'] = test_metrics['text'].apply(get_adverbs)

/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
test.to_csv('test_features.csv', index=False)

### Word2Vec

#### Keywords

In [13]:
tweets['keyword'] = tweets['keyword'].fillna('NULL')
test['keyword'] = test['keyword'].fillna('NULL')

In [14]:
keywords = tweets[['keyword', 'id']]
keywords.head()

,keyword,id
0,NULL,1
1,NULL,4
2,NULL,5
3,NULL,6
4,NULL,7


In [15]:
keyword_tokens = keywords.keyword.unique().tolist()

In [16]:
keyword_tokens[0]

'NULL'

In [17]:
len(keyword_tokens)

222

In [18]:
from gensim.models import Word2Vec

# Parameters
# sg ({0, 1}, optional) - Training algorithm: 1 for skip-gram; otherwise CBOW.

keyword_vectors = Word2Vec([keyword_tokens], min_count=1, size= 100, workers=3, window =3, sg=1)

In [19]:
# Sanity check
keyword_vectors['ablaze']

/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([-2.9691348e-03, -4.6726349e-03,  2.6698408e-03,  1.3936218e-03,
       -4.0975353e-04, -2.2376438e-03, -4.5118001e-03,  3.5424023e-03,
       -1.2515603e-03,  1.1874196e-03,  4.9171234e-03, -2.1516369e-03,
        2.3249865e-03,  2.6831164e-03, -1.8474258e-03, -2.0176845e-03,
       -4.4094790e-03,  1.8744356e-03, -3.8899099e-03, -4.4326098e-03,
       -5.9894077e-04,  4.4174064e-03,  1.9389419e-03, -1.3160927e-03,
       -2.5508045e-03,  3.7587201e-03, -9.0516126e-04, -1.0121947e-03,
        2.1766676e-03, -1.9006715e-03, -3.4770861e-03, -3.4123656e-04,
       -3.4737966e-03, -1.8389890e-03, -4.8271392e-04, -2.9958151e-03,
       -2.6793319e-03, -9.7406458e-04, -3.2200413e-03,  4.3561123e-03,
        5.7204859e-05,  3.0434027e-03,  2.6703451e-03,  1.8030163e-03,
        1.7714408e-03, -4.0001585e-03, -2.8440841e-03,  1.4819726e-03,
       -4.0566339e-03,  4.1848528e-03,  2.5892959e-03, -2.2633339e-03,
        3.3276931e-03,  8.0631132e-04, -6.7944662e-04,  1.7627685e-03,
      

In [20]:
to_vector_matrix = {}

for k in keyword_tokens:
    to_vector_matrix[k] = keyword_vectors[k]

/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [21]:
keyword_w2v = pd.DataFrame.from_dict(to_vector_matrix).T.reset_index()
keyword_w2v

,index,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,NULL,-0.002675,-0.001897,-0.000787,-0.001888,0.003148,0.002227,-0.001302,-0.003276,-0.004409,...,-0.002599,-0.004922,0.001684,0.004708,-0.004160,0.003790,-0.003561,0.003940,0.000939,-0.002726
1,ablaze,-0.002969,-0.004673,0.002670,0.001394,-0.000410,-0.002238,-0.004512,0.003542,-0.001252,...,0.002128,0.003162,0.004215,0.004947,-0.000472,0.003841,0.003310,0.002058,0.003145,0.002920
2,accident,-0.003366,0.002198,0.000095,0.003895,0.001833,0.001408,0.002246,-0.003294,0.000644,...,0.002074,0.000825,0.000442,-0.004813,-0.004949,0.003789,0.003702,-0.000228,0.004167,0.004664
3,aftershock,0.001413,0.003539,-0.002217,-0.003810,-0.004349,-0.004274,-0.001596,0.000768,0.003827,...,0.002345,-0.002228,0.000241,0.001078,0.002260,0.004903,-0.002628,0.001688,-0.000927,-0.000476
4,airplane%20accident,0.000383,0.000226,0.004611,0.003410,0.003898,-0.004922,0.000915,0.001955,0.001060,...,0.003806,0.001122,0.004927,-0.002005,0.002223,0.004315,-0.000272,0.001431,-0.004914,0.000174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,wounded,0.001823,-0.003264,0.004590,0.004210,0.002555,0.000461,0.004600,0.002716,0.003216,...,-0.000338,-0.004431,-0.000998,0.004568,0.001108,0.000484,-0.001134,0.000524,0.000014,-0.003300
218,wounds,-0.001801,0.003490,0.003534,-0.001836,-0.000305,0.002193,0.004454,0.002216,-0.002202,...,0.004561,-0.001985,0.000687,-0.003406,-0.004135,0.004414,-0.001768,-0.002348,-0.003370,-0.004599
219,wreck,0.001305,0.001566,-0.004231,0.003320,-0.001554,0.002129,-0.003257,-0.002014,-0.000336,...,0.002939,0.003866,-0.001899,0.003966,0.001296,-0.003790,0.001715,-0.000759,-0.000663,-0.003983
220,wreckage,0.004178,0.000893,0.003457,0.000586,0.000962,-0.003000,-0.003203,0.003771,-0.003912,...,-0.001317,-0.000308,0.001555,-0.000482,-0.004597,0.003271,0.003350,0.002725,-0.004631,0.004226


In [22]:
aux = []
aux.append('keyword')
for i in range (0, 100):
    name = 'v' + str(i)
    aux.append(name)
len(aux)

101

In [23]:
keyword_w2v.columns = aux

In [24]:
keyword_w2v.head()

,keyword,v0,v1,v2,v3,v4,v5,v6,v7,v8,...,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99
0,NULL,-0.002675,-0.001897,-0.000787,-0.001888,0.003148,0.002227,-0.001302,-0.003276,-0.004409,...,-0.002599,-0.004922,0.001684,0.004708,-0.004160,0.003790,-0.003561,0.003940,0.000939,-0.002726
1,ablaze,-0.002969,-0.004673,0.002670,0.001394,-0.000410,-0.002238,-0.004512,0.003542,-0.001252,...,0.002128,0.003162,0.004215,0.004947,-0.000472,0.003841,0.003310,0.002058,0.003145,0.002920
2,accident,-0.003366,0.002198,0.000095,0.003895,0.001833,0.001408,0.002246,-0.003294,0.000644,...,0.002074,0.000825,0.000442,-0.004813,-0.004949,0.003789,0.003702,-0.000228,0.004167,0.004664
3,aftershock,0.001413,0.003539,-0.002217,-0.003810,-0.004349,-0.004274,-0.001596,0.000768,0.003827,...,0.002345,-0.002228,0.000241,0.001078,0.002260,0.004903,-0.002628,0.001688,-0.000927,-0.000476
4,airplane%20accident,0.000383,0.000226,0.004611,0.003410,0.003898,-0.004922,0.000915,0.001955,0.001060,...,0.003806,0.001122,0.004927,-0.002005,0.002223,0.004315,-0.000272,0.001431,-0.004914,0.000174


In [26]:
keyword_w2v.to_csv('keyword_features.csv', index=False)

#### Locations (proximamente)